In [1]:
!rm -r ../data/logs/src_tb_logs

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext tensorboard

import os
cpu='0'
os.environ['CUDA_VISIBLE_DEVICES']=cpu

from bio_learn2 import *

In [3]:
(train_X, train_y), (test_X, test_y) = get_data('train'), get_data('test')
print('train_data, test_data:', train_X.shape, train_y.shape, test_X.shape, test_y.shape)

train_data, test_data: torch.Size([60000, 784]) torch.Size([60000]) torch.Size([10000, 784]) torch.Size([10000])


In [4]:
file = '../data/0020_unsupervised_weights.pkl'
if os.path.isfile(file): weights = torch.load(file)
else: 
    weights = get_unsupervised_weights(train_X, n_hidden=2000, n_epochs=200, batch_size=3584)
    torch.save(weights, file)
tb.add_histogram('weights-hist', weights)
tb.add_figure('weights', draw_unsupervised_weights(weights, 10, 10, 28))

In [6]:
n_classes = 10
model = BioClassifier(weights, n_classes).cuda()
tb.add_text('model-bio', 'BioClassifier')
tb.add_graph(model, input_to_model=torch.randn(1, 28*28).cuda())
run_test(train_X, train_y, test_X, test_y, model, epochs=10, loss=BioLoss(n_classes), batch_size=1024, lr=1e-4)  
# Final Accuracy: 0.97 Took: 55s

Final Accuracy: 0.92 Took: 12s


In [ ]:
model_file = '../data/0020_BioClassifier.pkl'
torch.save(model, model_file)

In [ ]:
model2 = SimpleConvNet(n_classes)
tb.add_text('model-cnn', 'SimpleConvNet')
tb.add_graph(model2, input_to_model=torch.randn(1, 28*28).cuda())
run_test(train_X, train_y, test_X, test_y, model2, epochs=100, loss=torch.nn.NLLLoss(), batch_size=1024, lr=1e-4)  
# Final Accuracy: 0.98 Took: 95s

In [ ]:
model_file_2 = '../data/0020_SimpleConvClassifier.pkl'
torch.save(model2, model_file_2)